<h1><center>Main Notebook - Logistic Regression</center></h1>

# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2
from utils import *
import utils.transformers.transformers as tr
import utils.transformers.sk4pandas as s4p
import utils.transformers.knimp as knp

import os
import pickle
import urllib.request
import warnings

import git
from IPython.core.magic import register_cell_magic
import mlflow
import pandas as pd
from sklearn import set_config
from sklearn.base import clone
from sklearn.compose import make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, loguniform

warnings.filterwarnings('ignore')
set_config(display='text')

git_repo = git.Repo(os.getcwd())

@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

# Init Datasets

In [2]:
raw_main_df = pd.read_csv('data/train.csv')
raw_sub_df = pd.read_csv('data/test.csv')
raw_train_df, raw_test_df = train_test_split(raw_main_df, test_size=100, random_state=42)

X_all, y_all = raw_main_df.drop('Survived', axis=1), raw_main_df['Survived']
X_train, y_train = raw_train_df.drop('Survived', axis=1), raw_train_df['Survived']
X_test, y_test = raw_test_df.drop('Survived', axis=1), raw_test_df['Survived']

raw_train_df.sample(5, random_state=42)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
678,679,0,3,"Goodwin, Mrs. Frederick (Augusta Tyler)",female,43.0,1,6,CA 2144,46.9000,NaN,S
49,50,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.0,1,0,349237,17.8000,NaN,S
691,692,1,3,"Karun, Miss. Manca",female,4.0,0,1,349256,13.4167,NaN,C
499,500,0,3,"Svensson, Mr. Olof",male,24.0,0,0,350035,7.7958,NaN,S
303,304,1,2,"Keane, Miss. Nora A",female,NaN,0,0,226593,12.3500,E101,Q


# Prepare Pipeline

In [3]:
%%run_and_save blueprints/pipeline.py

clip_outliers = [(
    'Float',
    tr.ClipOutliers(std_band=3),
    make_column_selector(dtype_include=['float64'])
)]

fillna_constant = [(
    'Cabin',
    s4p.SimpleImputer(strategy='constant', fill_value='UNKNOWN'),
    ['Cabin']
)]


steps_main = [
    ('Set working columns', tr.SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket']) ),
    ('Format Cabins', tr.Cabin() ),
    ('Standardize', s4p.StandardScaler(cols_select=['Age', 'Fare']) ),
    ('Clip Outliers', tr.ClipOutliers(cols_select=['Age', 'Fare'], std_band=3)),
    ('Replace NaNs With Constants', s4p.ColumnTransformer(fillna_constant, remainder='passthrough') ),
    ('Prepare coltypes for imputation', tr.AsTypes() ),
    ('Impute NaNs left with KNN', knp.KNImp(min_frequency=20) ),
    ('OneHot Encoding', s4p.OneHotEncoder(cols_select=['Cabin', 'Parch', 'Pclass', 'Sex', 'SibSp', 'Embarked']) ),
    ('Logistic Regression', LogisticRegression())
]

pipe = Pipeline(steps_main)

# Grid Params

In [4]:
%%run_and_save blueprints/grid_params.py


ml_params_distributions = {
    'Logistic Regression__C': uniform(1, 10),
}

rs = RandomizedSearchCV (
    pipe,
    param_distributions = ml_params_distributions,
    n_iter=10,
    n_jobs=-1,
    refit=True,
    cv=CVSplitter(5, 80),
    return_train_score=True
)

# ML Flow Run

In [ ]:
mlflow.sklearn.autolog(max_tuning_runs=3)

with mlflow.start_run(run_name='Logistic Regression - After KNN Imputer') as run:
    mlflow.set_tags(get_lastcommit_infos(git_repo))
    mlflow.log_artifact('blueprints/pipeline.py', 'blueprints')
    mlflow.log_artifact('blueprints/grid_params.py', 'blueprints')
    #pipe.fit(X_all, y_all)
    rs.fit(X_all, y_all)

# Refit best model

In [6]:
run_id = '757041383f924d9f9b5ad609c45248eb'
# url_local_model = f'file:///Users/i538262/Desktop/Developments/PERSO/KAGGLE%20Titanic/mlruns/0/{run_id}/artifacts/model/model.pkl' # model
url_local_model = f'file:///Users/i538262/Desktop/Developments/PERSO/KAGGLE%20Titanic/mlruns/0/{run_id}/artifacts/best_estimator/model.pkl' # best estimator (from rs)

with urllib.request.urlopen(url_local_model) as model_file:
    best_model = pickle.load(model_file)
    
best_model_allfit = clone(best_model)
best_model_allfit.fit(X_all, y_all)

2022/05/25 23:02:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c29f5bcd3d024cd3ab025cfbdc2290b7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/05/25 23:02:12 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('Set working columns', SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket'])), ('Format Cabins', Cabin()), ('Standardize', StandardScaler(cols_ignore=['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin',
                            'Embarked'],
  ...`
2022/05/25 23:02:12 WARNING mlflow.utils: Truncated the value of the key `Replace NaNs With Constants`. Truncated value: `ColumnTransformer(Cabin__cols_ignore=[], Cabin__cols_select=[],
                  Cabin__fill_value='UNKNOWN', Cabin__strategy='constant',
                  transformers=[('Cabin',
                                 SimpleImputer(fill_value='UNKNOWN...`


<KNImputer> Cols that will be OneHot encoded :
	- Cabin {'A', 'F', 'G', 'E', 'T', 'D', 'UNKNOWN', 'B', 'C'}
	- Pclass {1, 2, 3}
	- Sex {'female', 'male'}
	- SibSp {0, 1, 2, 3, 4, 5, 8}
	- Parch {0, 1, 2, 3, 4, 5, 6}
	- Embarked {nan, 'Q', 'C', 'S'}


Pipeline(steps=[('Set working columns',
                 SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket'])),
                ('Format Cabins', Cabin()),
                ('Standardize',
                 StandardScaler(cols_ignore=['Pclass', 'Sex', 'SibSp', 'Parch',
                                             'Cabin', 'Embarked'],
                                cols_select=['Age', 'Fare'])),
                ('Clip Outliers',
                 ClipOutliers(cols_ignore=['Pclass', 'Sex', 'SibSp', 'Parch',
                                           'Cabin', 'Embarked'],
                              cols_select=...
                                                  ['Cabin'])])),
                ('Prepare coltypes for imputation', AsTypes()),
                ('Impute NaNs left with KNN',
                 KNImp(cols_onehot=['Cabin', 'Pclass', 'Sex', 'SibSp', 'Parch',
                                    'Embarked'],
                       min_frequency=20)),
                ('OneHot

# Submit models predictions

In [7]:
branch_name = get_lastcommit_infos(git_repo)['Branch']
submit(f'submissions/{branch_name}/{run_id}.csv', raw_sub_df.PassengerId, best_model_allfit.predict(raw_sub_df))

# Model insights

In [ ]:
for c, n in zip(best_model_allfit[-1].coef_[0], best_model_allfit[-1].feature_names_in_):
    print(n, c)

# Code tests